In [1]:
import polars as pl
from huggingface_hub import snapshot_download
from datasets import load_dataset
import os
import sys
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
sys.path.append("..")
%load_ext autoreload
%autoreload 2


/Users/brianfitzgerald/Documents/GitHub/smolmodels/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
gutenberg_books = pl.read_parquet("../dataset_files/gutenberg_books.parquet", n_rows=10)

In [20]:
import tiktoken
from synthetic_data.gutenberg_parser import super_cleaner
from synthetic_data.tasks.next_chapter import filter_valid_paragraphs, find_chunks_with_summary

encoder = tiktoken.get_encoding("o200k_base")


paragraphs = super_cleaner(gutenberg_books["text"][7])
chunks = filter_valid_paragraphs(paragraphs, 15)
for c in chunks:
    print(c)
    print("-" * 100)


On starless nights when the moon was obscured by the earth's shadow,
Pierre Soret walked alone down the darkest and most deserted streets in
London. He did this to avoid the people who might look into his face.
His face! He shuddered, his pinched shoulders wrenching sharply with
a bitter shrug. Could anyone call this mask, this unearthly mass of
bubbling flesh, boiling and seething in his own blood, a face?
----------------------------------------------------------------------------------------------------
Pierre knew what always happened when people looked into his
torture-shattered eyes. The sight of their horror sickened him. His
memory ached with the sharpness of pain he had felt on those few
occasions when some luckless persons had inadvertently seen, and
halted, frozen momentarily with the intense horror and nausea that
overwhelmed them, their faces graven with revulsion; a moment later to
totter rapidly past him down the street, the tension of their feeling
released with an effo